### telegram을 이용한 챗봇 생성
* 5초에 한번씩 updates 내용 확인하여 새로운 대화 내용이 생겼을 때, 
* text에 요약이라는 단어가 들어있으면 
* 요약을 제외한 나머지 text로 워드 클라우드 생성해서 이미지 보내기
* 핸드폰에서 종료를 전송하면 챗봇 종료
* 워드 클라우드의 키워드에서(자연어 처리) 형용사를 추출하게 되면 감성분석

In [ ]:
# 뉴스 제목 추출 함수
def news_get(keyword):
    query = parse.quote(keyword) # 요청할 키워드 인코딩
    # rss 형식 파싱
    feeds = feedparser.parse(f'https://news.google.com/rss/search?q={query}&hl=ko&gl=KR&ceid=KR%3Ako')
    
    # 뉴스 제목 추출 
    news_title_list = [feed['title'] for feed in feeds['entries']]
    news_text = ''
    for title in news_title_list:
        news_text += title
    
    return news_text

In [ ]:
# 문자열로 워드 클라우드 파일 만들기 - wc_gen(문자열) 함수
# word cloud 생성 함수
def generate_wordcloud(words, mask, keyword): 
    font_path = "c:/Windows/Fonts/malgun.ttf"
    
    # word cloud 초기값
    wordcloud = WordCloud(
        font_path = font_path,
        mask = mask,
        background_color='white',
#         width = heart_mask.shape[1],
#         height = heart_mask.shape[0],
        width = 3840,
        height = 2160,
        stopwords=[keyword] # stopwords를 사용하여 keword를 제외한 나머지 키워드 추출
    )
    
    wordcloud = wordcloud.generate_from_frequencies(words)
    fig = plt.figure(figsize=(10,10), facecolor='white', edgecolor='blue')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show() 
    
    # 이미지 저장, 경로
    fig.savefig(f'WordCloudImage/{keyword}_wordcloud.png')
    img_path = f'WordCloudImage/{keyword}_wordcloud.png'
    
    return img_path


def wc_get(news_text, keyword):
    nlpy = Okt()
    nouns = nlpy.nouns(news_text)
    
    nouns = [n for n in nouns if len(n) > 1]
    
    count = Counter(nouns)
    tags = count.most_common(500)
    
    # wordcloud에서 언론사 제외하기 위해 tags에서 언론사 제외    
    journal_list = []
    for tag in tags:
        if '일보' in tag[0]:
            journal_list.append(tag)
            tags.remove(tag)
        elif '신문' in tag[0]:
            journal_list.append(tag)
            tags.remove(tag)
        elif '뉴스' in tag[0]:
            journal_list.append(tag)
            tags.remove(tag)
        elif '투데이' in tag[0]: 
            journal_list.append(tag)
            tags.remove(tag)
        elif '데일리' in tag[0]:
            journal_list.append(tag)
            tags.remove(tag)
        elif '경제' in tag[0]:
            journal_list.append(tag)
            tags.remove(tag)
            
    print('tags',tags)
    print('journal', journal_list)
    
    # word cloud mask 적용
    heart_mask = np.array(Image.open('WordCloudImage/heart_mask.jpg'))   
    
    return generate_wordcloud(dict(tags), heart_mask, keyword)

In [ ]:
# python-telegram-bot 이용하여 나에게(내 폰으로) 메시지나 이미지 보내기 - send_msg(메시지), send_img(이미지) 함수
import telegram

def send_text_img(keyword_text, img_path):
    # chat key : 1414946658
    my_token = '1340754754:AAESxOYIBqKOkRmA-mPKcFf29faZlG8HCFE'
    bot = telegram.Bot(token = my_token)
    
    bot.send_message(chat_id='1414946658', text=f'{keyword_text} wc image')
    # 이미지 경로 open하여 메세지로 전송
    bot.sendPhoto(chat_id='1414946658', photo=open(img_path, 'rb'))
    
    #bot 정보 추출
    updates = bot.getUpdates()
    
    # telegram bot 메세지 이력 
    for u in updates:
        print(u)

In [ ]:
# 5초에 한번씩 updates 내용 확인
# 새로운 대화 내용 존재 여부
# text에서 요약 확인
# ? > 5초에 한번씩 update 내용 확인, 새로운 대화 내용 존재 여부
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler
import time

def chat_bot():
    my_token = '1340754754:AAESxOYIBqKOkRmA-mPKcFf29faZlG8HCFE'
    bot = telegram.Bot(token = my_token)
    updates = bot.getUpdates()
    latest_updates_num = len(updates)    
    
    # 요약이라는 키워드가 포함되면 해당 키워드와 관련된 워드 클라우드 생성하여 이미지 전송
    # 종료라는 키워드가 포함되면 챗봇 종료
    while True:    
        updates = bot.getUpdates()
        post_updates_num = len(updates)

        if latest_updates_num != post_updates_num:
        # 가장 최근의 chat 내용 가져오기
            latest_chat_text = updates[-1].message.text
            # chat text에 요약이라는 단어 들어있는지 확인
            if '요약' in latest_chat_text:
                keyword = latest_chat_text.replace('요약','').strip()
                
                news = news_get(keyword) # keyword로 구글 검색 후 타이틀 추출
                img_path = wc_get(news, keyword) # 추출한 타이틀 자연어 처리 후 워드클라우드 생성
                send_text_img(keyword, img_path) # telegram 봇 메시지 전송
            elif '종료' in latest_chat_text:
                break
            
            latest_updates_num = len(updates)
            
        # 5초에 한번씩 함수 실행
        time.sleep(5)
        
chat_bot()